In [ ]:
!pip install faiss-gpu
!pip install datasets
import torch
import faiss
import numpy as np
import pandas as pd

from google.colab import drive
drive.mount('/content/drive')

# Change to directory
%cd '/content/drive/MyDrive/CS 7650 project'

In [2]:
from torch.nn.functional import softmax
from datasets import Dataset
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

# Run KNN inference

In [3]:
'''
  datastore: a dataset object mapping from embeddings to labels
  newData: a new embedding to run KNN inference on
  labels: possible labels, e.g. [0, 1]
  k: number of neighbors in KNN
  T: temperature
'''
def KNNinference_onesample(datastore, newData, labels, k=64, T=10):
  scores, samples = datastore.get_nearest_examples("embeddings", newData, k=k)
  samples_df = pd.DataFrame.from_dict(samples)
  samples_df["scores"] = scores
  samples_df.sort_values("scores", ascending=False, inplace=True)
  probs = softmax(torch.Tensor(samples_df['scores'].values / T), dim=0)
  ret = np.zeros((1, len(labels)))
  idx = samples_df['label'].values
  for i in range(len(idx)):
    ret[0, idx[i]] += probs[i]
  return np.argmax(ret)

In [4]:
def KNNinference(data, gt, inf_data, k=64, T=10):
  embeddings_dataset = torch.load(data).numpy().tolist()
  label_dataset = torch.load(gt).numpy().tolist()
  data_store = Dataset.from_dict({"embeddings": embeddings_dataset, "label": label_dataset})
  data_store.add_faiss_index(column="embeddings")
  labels = np.unique(label_dataset)
  question_embedding = torch.load(inf_data).numpy()
  n = question_embedding.shape[0]
  result = np.zeros(n)
  for i in range(question_embedding.shape[0]):
    result[i] = KNNinference_onesample(data_store, question_embedding[i], labels, k=k, T=T)
  return result, labels

In [5]:
base_path = "./data/"
dataset_path = ["Constraint_", "Liar_", "Political_", "Rumor_"]
train_path = "Train_"
val_path = "Validation_"
test_path = "Test_"
mode_path = ["pretrain_", "finetune_"]
emb_path = "emb.pt"
gt_path = "gt.pt"
pred_path = "pred.pt"

In [6]:
k_choices = [4, 8, 16, 32, 64]
T_choices = [0.01, 0.1, 1, 10]

In [7]:
def hyper_parameters_tuning(data, gt, inf_data, inf_gt, k_choices, T_choices):
  results = np.zeros((len(k_choices), len(T_choices)))
  for i in range(len(k_choices)):
    for j in range(len(T_choices)):
      pred, _ = KNNinference(data, gt, inf_data, k=k_choices[i], T=T_choices[j])
      y_true = torch.load(inf_gt).numpy()
      results[i, j] = accuracy_score(y_true, pred)
  choice = np.unravel_index(results.argmax(), results.shape)
  return k_choices[choice[0]], T_choices[choice[1]], np.max(results)

In [8]:
def evaluate(inf_gt, pred, labels):
  y_true = torch.load(inf_gt).numpy()
  return accuracy_score(y_true, pred), f1_score(y_true, pred, labels=labels, average='weighted')

# Hyper parameter tuning k and T



In [9]:
data = dataset_path[0]
for mode in mode_path:
  data_path = base_path + data + train_path + mode + emb_path
  gt_path_full = base_path + data + train_path + mode + gt_path
  inf_data_path = base_path + data + val_path + mode + emb_path
  inf_gt_path = base_path + data + val_path + mode + gt_path

  k, T, acc = hyper_parameters_tuning(data_path, gt_path_full, inf_data_path, inf_gt_path, k_choices, T_choices)
  print("For " + data[:-1] + " dataset under setting " + mode[:-1])
  print("optimal k: " + str(k))
  print("optimal T: " + str(T))

  pred, labels = KNNinference(data_path, gt_path_full, inf_data_path, k=k, T=T)
  acc, f1 = evaluate(inf_gt_path, pred, labels)
  print("validation acc: " + str(acc) + " f1: " + str(f1))

  inf_data_path = base_path + data + test_path + mode + emb_path
  inf_gt_path = base_path + data + test_path + mode + gt_path

  pred, labels = KNNinference(data_path, gt_path_full, inf_data_path, k=k, T=T)
  acc, f1 = evaluate(inf_gt_path, pred, labels)
  print("test acc: " + str(acc) + " f1: " + str(f1))

  0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

...

For Constraint dataset under setting finetune
optimal k: 32
optimal T: 0.01


  0%|          | 0/7 [00:00<?, ?it/s]

validation acc: 0.9733644859813084 f1: 0.9733559182085807


  0%|          | 0/7 [00:00<?, ?it/s]

test acc: 0.9682242990654205 f1: 0.9682182888661518


In [11]:
data = dataset_path[1]
for mode in mode_path:
  data_path = base_path + data + train_path + mode + emb_path
  gt_path_full = base_path + data + train_path + mode + gt_path
  inf_data_path = base_path + data + val_path + mode + emb_path
  inf_gt_path = base_path + data + val_path + mode + gt_path

  k, T, acc = hyper_parameters_tuning(data_path, gt_path_full, inf_data_path, inf_gt_path, k_choices, T_choices)
  print("For " + data[:-1] + " dataset under setting " + mode[:-1])
  print("optimal k: " + str(k))
  print("optimal T: " + str(T))

  pred, labels = KNNinference(data_path, gt_path_full, inf_data_path, k=k, T=T)
  acc, f1 = evaluate(inf_gt_path, pred, labels)
  print("validation acc: " + str(acc) + " f1: " + str(f1))

  inf_data_path = base_path + data + test_path + mode + emb_path
  inf_gt_path = base_path + data + test_path + mode + gt_path

  pred, labels = KNNinference(data_path, gt_path_full, inf_data_path, k=k, T=T)
  acc, f1 = evaluate(inf_gt_path, pred, labels)
  print("test acc: " + str(acc) + " f1: " + str(f1))

  0%|          | 0/11 [00:00<?, ?it/s]

  0%|          | 0/11 [00:00<?, ?it/s]

  0%|          | 0/11 [00:00<?, ?it/s]

  0%|          | 0/11 [00:00<?, ?it/s]

  0%|          | 0/11 [00:00<?, ?it/s]

...

For Liar dataset under setting finetune
optimal k: 8
optimal T: 10


  0%|          | 0/11 [00:00<?, ?it/s]

validation acc: 0.2562305295950156 f1: 0.2542691935223387


  0%|          | 0/11 [00:00<?, ?it/s]

test acc: 0.2509865824782952 f1: 0.25119072421424654


In [10]:
data = dataset_path[2]
for mode in mode_path:
  data_path = base_path + data + train_path + mode + emb_path
  gt_path_full = base_path + data + train_path + mode + gt_path
  inf_data_path = base_path + data + val_path + mode + emb_path
  inf_gt_path = base_path + data + val_path + mode + gt_path

  k, T, acc = hyper_parameters_tuning(data_path, gt_path_full, inf_data_path, inf_gt_path, k_choices, T_choices)
  print("For " + data[:-1] + " dataset under setting " + mode[:-1])
  print("optimal k: " + str(k))
  print("optimal T: " + str(T))

  pred, labels = KNNinference(data_path, gt_path_full, inf_data_path, k=k, T=T)
  acc, f1 = evaluate(inf_gt_path, pred, labels)
  print("validation acc: " + str(acc) + " f1: " + str(f1))

  inf_data_path = base_path + data + test_path + mode + emb_path
  inf_gt_path = base_path + data + test_path + mode + gt_path

  pred, labels = KNNinference(data_path, gt_path_full, inf_data_path, k=k, T=T)
  acc, f1 = evaluate(inf_gt_path, pred, labels)
  print("test acc: " + str(acc) + " f1: " + str(f1))

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

...

For Political dataset under setting finetune
optimal k: 4
optimal T: 10


  0%|          | 0/1 [00:00<?, ?it/s]

validation acc: 0.6363636363636364 f1: 0.6375291375291375


  0%|          | 0/1 [00:00<?, ?it/s]

test acc: 0.6222222222222222 f1: 0.6376185364108069


In [12]:
data = dataset_path[3]
for mode in mode_path:
  data_path = base_path + data + train_path + mode + emb_path
  gt_path_full = base_path + data + train_path + mode + gt_path
  inf_data_path = base_path + data + val_path + mode + emb_path
  inf_gt_path = base_path + data + val_path + mode + gt_path

  k, T, acc = hyper_parameters_tuning(data_path, gt_path_full, inf_data_path, inf_gt_path, k_choices, T_choices)
  print("For " + data[:-1] + " dataset under setting " + mode[:-1])
  print("optimal k: " + str(k))
  print("optimal T: " + str(T))

  pred, labels = KNNinference(data_path, gt_path_full, inf_data_path, k=k, T=T)
  acc, f1 = evaluate(inf_gt_path, pred, labels)
  print("validation acc: " + str(acc) + " f1: " + str(f1))

  inf_data_path = base_path + data + test_path + mode + emb_path
  inf_gt_path = base_path + data + test_path + mode + gt_path

  pred, labels = KNNinference(data_path, gt_path_full, inf_data_path, k=k, T=T)
  acc, f1 = evaluate(inf_gt_path, pred, labels)
  print("test acc: " + str(acc) + " f1: " + str(f1))

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

...

For Rumor dataset under setting finetune
optimal k: 8
optimal T: 10


  0%|          | 0/6 [00:00<?, ?it/s]

validation acc: 0.7158774373259053 f1: 0.7110569021376006


  0%|          | 0/6 [00:00<?, ?it/s]

test acc: 0.7346796657381616 f1: 0.733693646094041


In [13]:
# "Constraint_", "Liar_", "Political_", "Rumor_"
optim_k = np.array([[8, 32],
                    [64, 8],
                    [8, 4],
                    [8, 8]])
optim_T = np.array([[10, 0.01],
                    [1, 10],
                    [0.01, 10],
                    [10, 10]])

# Hyper parameter tuning $λ$

In [14]:
'''
  datastore: a dataset object mapping from embeddings to labels
  newData: a new embedding to run KNN inference on
  labels: possible labels, e.g. [0, 1]
  k: number of neighbors in KNN
  T: temperature
'''
def KNNinference_onesample_alpha(datastore, newData, labels, k=64, T=10):
  scores, samples = datastore.get_nearest_examples("embeddings", newData, k=k)
  samples_df = pd.DataFrame.from_dict(samples)
  samples_df["scores"] = scores
  samples_df.sort_values("scores", ascending=False, inplace=True)
  probs = softmax(torch.Tensor(samples_df['scores'].values / T), dim=0)
  ret = np.zeros((1, len(labels)))
  idx = samples_df['label'].values
  for i in range(len(idx)):
    ret[0, idx[i]] += probs[i]
  return ret

In [15]:
def KNNinference_alpha(data, gt, inf_data, k=64, T=10):
  embeddings_dataset = torch.load(data).numpy().tolist()
  label_dataset = torch.load(gt).numpy().tolist()
  data_store = Dataset.from_dict({"embeddings": embeddings_dataset, "label": label_dataset})
  data_store.add_faiss_index(column="embeddings")
  labels = np.unique(label_dataset)
  question_embedding = torch.load(inf_data).numpy()
  n = question_embedding.shape[0]
  result = np.zeros((n, len(labels)))
  for i in range(question_embedding.shape[0]):
    result[i] = KNNinference_onesample_alpha(data_store, question_embedding[i], labels, k=k, T=T)
  return result, labels

In [16]:
def alpha_tuning(data, gt, bert_output, inf_data, inf_gt, k, T, alphas):
  results = np.zeros(len(alphas))
  pred, _ = KNNinference_alpha(data, gt, inf_data, k=k, T=T)
  y_true = torch.load(inf_gt).numpy()
  bert_pred = softmax(torch.load(bert_output), dim=1)
  for i in range(len(alphas)):
    comb_pred = alphas[i] * pred + (1 - alphas[i]) * bert_pred.numpy()
    pred_labels = np.argmax(comb_pred, axis=1)
    results[i] = accuracy_score(y_true, pred_labels)
  choice = np.unravel_index(results.argmax(), results.shape)
  return alphas[choice[0]]

In [17]:
def alpha_inference(data, gt, bert_output, inf_data, k=64, T=10, alpha=0):
  pred, labels = KNNinference_alpha(data, gt, inf_data, k, T)
  bert_pred = softmax(torch.load(bert_output), dim=1)
  comb_pred = alpha * pred + (1 - alpha) * bert_pred.numpy()
  result = np.argmax(comb_pred, axis=1)
  return result, labels

In [18]:
alphas = np.arange(0, 1, 0.1)

In [19]:
for i in range(len(dataset_path)):
  data = dataset_path[i]
  j = 1
  mode = mode_path[1]

  data_path = base_path + data + train_path + mode + emb_path
  gt_path_full = base_path + data + train_path + mode + gt_path
  pred_path_full = base_path + data + val_path + mode + pred_path
  inf_data_path = base_path + data + val_path + mode + emb_path
  inf_gt_path = base_path + data + val_path + mode + gt_path

  k = int(optim_k[i, j])
  T = optim_T[i, j]
  alpha = alpha_tuning(data_path, gt_path_full, pred_path_full, inf_data_path, inf_gt_path, k, T, alphas)
  print("For " + data[:-1] + " dataset under setting " + mode[:-1])
  print("optimal k: " + str(k))
  print("optimal T: " + str(T))
  print("optimal lambda: " + str(alpha))

  pred, labels = alpha_inference(data_path, gt_path_full, pred_path_full, inf_data_path, k, T, alpha)
  acc, f1 = evaluate(inf_gt_path, pred, labels)
  print("validation acc: " + str(acc) + " f1: " + str(f1))

  pred_path_full = base_path + data + test_path + mode + pred_path
  inf_data_path = base_path + data + test_path + mode + emb_path
  inf_gt_path = base_path + data + test_path + mode + gt_path

  pred, labels = alpha_inference(data_path, gt_path_full, pred_path_full, inf_data_path, k, T, alpha)
  acc, f1 = evaluate(inf_gt_path, pred, labels)
  print("test acc: " + str(acc) + " f1: " + str(f1))

  0%|          | 0/7 [00:00<?, ?it/s]

For Constraint dataset under setting finetune
optimal k: 32
optimal T: 0.01
optimal lambda: 0.5


  0%|          | 0/7 [00:00<?, ?it/s]

validation acc: 0.9733644859813084 f1: 0.9733559182085807


  0%|          | 0/7 [00:00<?, ?it/s]

...

For Rumor dataset under setting finetune
optimal k: 8
optimal T: 10.0
optimal lambda: 0.6000000000000001


  0%|          | 0/6 [00:00<?, ?it/s]

validation acc: 0.7200557103064067 f1: 0.7149248337406903


  0%|          | 0/6 [00:00<?, ?it/s]

test acc: 0.7381615598885793 f1: 0.7375175200540312
